In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/vandahalasi/BRAIN2SPEECH_LesssGoo.git

In [ ]:
!git submodule init
!git submodule update

In [4]:
import LSTM
from create_dataloaders import get_data, create_datasets, create_dataloaders
from torch.optim import Adam
from torch.nn import MSELoss
from tqdm import tqdm

## Variables

In [ ]:
#Training variables
EPOCH = 10
Window_size = 1
Batch_size = 64
Learning_rate = 0.01
device = "gpu"

### Create dataloaders:

In [6]:
spectogram_train, spectogram_val, spectogram_test, features_train, features_val, features_test = get_data(r"drive/MyDrive/BRAIN2SPEECH/features")
# Only need to be run if train_stats.json is missing
# write_statistics_to_json(features_train)
train_dataset, val_dataset, test_dataset = create_datasets(spectogram_train, spectogram_val, 
        spectogram_test, features_train, features_val, features_test, window=Window_size)
train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, 
        test_dataset, batch_size=Batch_size)

## Training function

In [9]:
model = LSTM.SpectogramLSTM(input_size=4860, hidden_size=4860)
criterion = MSELoss()
optimizer = Adam(model.parameters(), lr=Learning_rate)

In [10]:
def train(EPOCH, model, train_loader, val_loader, criterion, optimizer):
    sum_loss = 0
    val_loss = 0
    for epoch in range(EPOCH):
        model.train()
        sum_loss = 0
        for in_t, target in tqdm(train_loader):
            outputs = model.forward(in_t)
            optimizer.zero_grad()
            
            loss = criterion(outputs, target)
            
            loss.backward()
            
            optimizer.step()
            sum_loss += loss.item()

        if epoch % 1 == 0:
            print(f"Epoch: {epoch}, loss: {sum_loss}") 
        
        for in_v, target_v in val_loader:
            outputs = model.forward(in_v)
            v_loss = criterion(outputs, target_v)
            val_loss += v_loss.item()

        print(f"Validation loss: {v_loss.item()}")

In [11]:
train(EPOCH, model, train_loader, val_loader, criterion, optimizer)

  0%|          | 4/1875 [03:13<25:10:20, 48.43s/it]


KeyboardInterrupt: 

## Test function

In [ ]:
def test(model, test_loader, criterion):
    sum_loss = 0
    for input_t, target in test_loader:
        outputs = model.forward(input_t)
        
        loss = criterion(outputs, target)
        sum_loss += loss.item()
    print(f"Test loss: {sum_loss}")
    return 

In [ ]:
test(model, test_loader, criterion)